# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [2]:
from langchain import hub
prompt = hub.pull("poet")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [3]:
prompt

StructuredPrompt(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'poet', 'lc_hub_commit_hash': '15675cccba351ece10bdc51d09fcbc03c37c273d70be9b5a3b2497d4a130875c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a professional poet and specialize in writing 2 line clever and sarcastic poems to any prompt given.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer.', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': [], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [4]:
hydrated_prompt = prompt.invoke({"question": "How is the weather?"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional poet and specialize in writing 2 line clever and sarcastic poems to any prompt given.', additional_kwargs={}, response_metadata={}), HumanMessage(content='How is the weather?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [5]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQBnSjJ2aX7m3rFGMBhMPD4SGmE7Q', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The forecast says rain, but I’m dressed for the sun,  \nSo I’ll just admire the tempest while I binge and have fun.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760357910, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=28, prompt_tokens=38, total_tokens=66, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [11]:
from langchain import hub
hub.pull("poet", include_model=True)

/opt/miniconda3/lib/python3.13/json/decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


StructuredPrompt(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'poet', 'lc_hub_commit_hash': '15675cccba351ece10bdc51d09fcbc03c37c273d70be9b5a3b2497d4a130875c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a professional poet and specialize in writing 2 line clever and sarcastic poems to any prompt given.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer.', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': [], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat

In [12]:
prompt

StructuredPrompt(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'poet', 'lc_hub_commit_hash': '15675cccba351ece10bdc51d09fcbc03c37c273d70be9b5a3b2497d4a130875c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a professional poet and specialize in writing 2 line clever and sarcastic poems to any prompt given.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer.', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': [], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Test out your prompt!

In [13]:
prompt.invoke({"question": "How is the weather"})

ChatPromptValue(messages=[SystemMessage(content='You are a professional poet and specialize in writing 2 line clever and sarcastic poems to any prompt given.', additional_kwargs={}, response_metadata={}), HumanMessage(content='How is the weather', additional_kwargs={}, response_metadata={})])

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [18]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("poet:b47ddda7", include_model=True)

/opt/miniconda3/lib/python3.13/json/decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


Run this commit!

In [22]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# Get just the prompt without the model binding to avoid structured output issues
from langchain import hub
prompt_only = hub.pull("poet")  # This gets just the prompt template

hydrated_prompt = prompt_only.invoke({"question": "What is run time polymorphism"})
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)

print(response.choices[0].message.content)

Runtime polymorphism in Java, often referred to as dynamic method dispatch, is a fundamental concept in object-oriented programming. It allows a method to perform different tasks based on the object that it is acting upon at runtime. This is typically achieved via method overriding and interfaces.

### How Runtime Polymorphism Works:

1. **Method Overriding**: This occurs when a subclass provides a specific implementation of a method that is already defined in its parent class. The method invoked is determined by the actual object type at runtime rather than the reference type.

2. **Reference Type**: A reference variable can point to an object of its own class or any subclass. When a method is called on that reference, Java checks which implementation to invoke based on the actual object type.

### Example of Runtime Polymorphism:

Here’s a simple example to illustrate runtime polymorphism:

```java
class Animal {
    void sound() {
        System.out.println("Animal makes a sound");


### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [26]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

hindi_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Hindi, make sure you only answer your users with Hindi.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

hindi_prompt_template = ChatPromptTemplate.from_template(hindi_prompt)
client.push_prompt("hindi-rag-prompt", object=hindi_prompt_template)

'https://smith.langchain.com/prompts/hindi-rag-prompt/d83fe14b?organizationId=6176107e-37d8-4611-acb8-9f68bf99512f'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [27]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

hindi_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak Hindi, make sure you only answer your users with Hindi.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

hindi_prompt_template = ChatPromptTemplate.from_template(hindi_prompt)
chain = hindi_prompt_template | model
client.push_prompt("hindi-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/hindi-runnable-sequence/5611dea6?organizationId=6176107e-37d8-4611-acb8-9f68bf99512f'